In [9]:
using Random
using Distributions
using FLoops
using Base.Threads
using BenchmarkTools

include("data.jl")
include("util.jl")

include("intervals/permutation.jl")
include("intervals/bootstrap.jl")
include("intervals/t.jl")

Threads.nthreads()

112

In [10]:
dtype = Float32
seed = 123

# DATA CONFIG

alpha = 0.05

# data
B  = 100   # num. coverage probabilities per boxplot
S  = 4300  # num. samples per coverage probability
nx = 15   # size of group 1
ny = 5    # size of group 2
MC = 10_000
perms = nothing
if binomial(nx + ny, nx) < 30_000
    perms = partition(nx, ny)
end

([1 1 … 5 6; 2 2 … 7 7; … ; 14 14 … 19 19; 15 16 … 20 20], [20 20 … 6 5; 19 19 … 4 4; … ; 17 17 … 2 2; 16 15 … 1 1])

In [12]:
# POPULATION SETTINGS

Random.seed!(123)

distrTypeX = Normal{dtype}
X_mu = random(Uniform(-1, 1), B)
X_sigma = random(Uniform(0.5, 1), B)
distrX = map(distrTypeX, X_mu, X_sigma)

distrTypeY = Normal{dtype}
Y_mu = X_mu
Y_sigma = random(Uniform(1.5, 2), B)
distrY = map(distrTypeY, Y_mu, Y_sigma);

In [13]:
deltas = @. mean(distrX) - mean(distrY)

@show distrX[1:2]
@show distrY[1:2]
@show deltas[1:2];

distrX[1:2] = Normal{Float32}[Normal{Float32}(μ=0.813f0, σ=0.977f0), Normal{Float32}(μ=-0.113f0, σ=0.923f0)]
distrY[1:2] = Normal{Float32}[Normal{Float32}(μ=0.813f0, σ=1.983f0), Normal{Float32}(μ=-0.113f0, σ=1.535f0)]
deltas[1:2] = Float32[0.0, 0.0]


In [14]:
Random.seed!(123)
xs = [dtype.(rand(distrX[i], nx, S)) for i in 1:B]
ys = [dtype.(rand(distrY[i], ny, S)) for i in 1:B]
@show size(ys)
ys[2]

size(ys) = (100,)


5×4300 Matrix{Float32}:
 -1.29209   -0.370489  -1.50338   …  -1.40419   -0.222648  -1.16133
 -3.08895   -0.177817  -0.627805     -0.515263   1.23733    0.500973
 -2.23605    0.100545   0.436316     -0.84021   -0.848975   0.451216
 -0.732024   0.53803   -0.187452      1.89326   -1.92684   -0.772154
  2.39267    1.68966   -0.917473      0.491966   0.941425   1.41847

In [15]:
# flatten into 3D matrix
X = reshape(hcat(xs...), nx, S, B)
Y = reshape(hcat(ys...), ny, S, B)
Y[:,:,2]

5×4300 Matrix{Float32}:
 -1.29209   -0.370489  -1.50338   …  -1.40419   -0.222648  -1.16133
 -3.08895   -0.177817  -0.627805     -0.515263   1.23733    0.500973
 -2.23605    0.100545   0.436316     -0.84021   -0.848975   0.451216
 -0.732024   0.53803   -0.187452      1.89326   -1.92684   -0.772154
  2.39267    1.68966   -0.917473      0.491966   0.941425   1.41847

In [16]:
function save_ci_results(results, methodId, B, S, pooled=nothing, two_sided=nothing; prefix="", dir="./")
    averages = []

    for batchId in 1:B
        batch = results[methodId, batchId, :]
        coverage = sum([j for (j, _) in batch]) / S
        width = sum([j for (_, j) in batch]) / S
        push!(averages, (coverage, width))
    end
    if isnothing(two_sided)
        save(averages, distrX[1:B], distrY[1:B], alpha, prefix=prefix, dir=dir)
    else
        alpha_ = two_sided ? alpha : alpha / 2
        save(averages, distrX[1:B], distrY[1:B], alpha_, pooled, two_sided, prefix=prefix, dir=dir)
    end
end

function save_permutation_results(results, B, S; prefix="", dir="./")
    i = 1
    per_method = []
    for two_sided in [true, false]
        for pooled in [true, false]
            save_ci_results(results, i, B, S, pooled, two_sided, prefix=prefix, dir=dir)
            i += 1
        end
    end
end

save_permutation_results (generic function with 1 method)

In [18]:
T = Threads.nthreads()
results = Array{Union{Nothing, Tuple}, 3}(undef, 6, B, S)
fill!(results, nothing)

#@time Threads.@threads for (i,j) in collect(Iterators.product(1:B, 1:S)) # 15.52 sec on (B,S) = (5, 1800)
@time @floop ThreadedEx(basesize=div(B*S, T)) for b in 1:B, s in 1:S
    @inbounds x = X[:,s,b]
    @inbounds y = Y[:,s,b]
 
    results[1, b, s] = permInterval(x, y, deltas[b], perms, true, alpha, twoSided, twoSided, 0.0005)    
    results[2, b, s] = permInterval(x, y, deltas[b], perms, false, alpha, twoSided, twoSided, 0.0005)
    results[3, b, s] = permInterval(x, y, deltas[b], perms, true, alpha/2, greater, smaller, 0.0005)
    results[4, b, s] = permInterval(x, y, deltas[b], perms, false, alpha/2, greater, smaller, 0.0005)
    
    """
    results[5, b, s] = bootstrap(x, y, deltas[b], alpha, nsamples=10_000)
    results[6, b, s] = tconf(x, y, deltas[b], alpha, false)
    """
end

dir = "../results/" * string(nx) * "_" * string(ny) * "/2/"
save_permutation_results(results, B, S; dir=dir)
#save_ci_results(results, 6, B, S; prefix="bs", dir=dir)

1450.995230 seconds (3.55 G allocations: 53.292 TiB, 54.01% gc time, 0.68% compilation time)
dir * filename = "../results/15_5/2/twoSided_pooled_0.05.csv"
dir * filename = "../results/15_5/2/twoSided_unpooled_0.05.csv"
dir * filename = "../results/15_5/2/oneSided_pooled_0.025.csv"
dir * filename = "../results/15_5/2/oneSided_unpooled_0.025.csv"
